## Equations of Motion Given

In [2]:
# Import everything we need from SymPy
import sympy as sym
import numpy as np
from sympy import Symbol, Matrix
from sympy import N as N_sympy
from sympy import simplify, sin, cos, init_printing
init_printing() # This function will make the outputs of SymPy look prettier and be easier to read

# Constants of the system
m_c = 12.0    # Mass of the chassis in kg
ixx_c = 1.0   # Roll mass moment of ineria of the chassis
iyy_c = 0.8   # Pitch mass moment of interia of the chassis
izz_c = 0.52  # Yaw mass moment of interia of the chassis

m_w = 1.2       # Mass of either wheel in kg
iaa_w = 0.0634  # Axial mass moment of interia of the wheels
itt_w = 0.0327  # Transverse mass moment of interia of the wheels
r_w = 0.325     # Radius of either wheel in meters

m = m_c + 2*m_w        # The total mass of the system
ixx = ixx_c + 2*itt_w  # The total roll mass moment of inertia of the system
iyy = iyy_c            # The total pitch mass moment of inertia of the system
izz = izz_c + 2*itt_w  # The total yaw mass moment of inertia of the system

h = 0.2   # Distance from the CoM of the chassis to wheel axle in meters
a = 0.35  # Half the distance between the wheels in meters

r_station = 20.0  # The radius of the station in meters
v_station = -0.1  # The angular velocity of the station in rad/second

# Create the system's generalized coordinates 
e_lat = Symbol('e_lat')          # The lateral error of the segbot. Positive is to the left of the target position.
phi = Symbol('phi')              # The yaw of the chassis of the segbot relative to the station. Positive is facing left.
phidot = Symbol('phidot')        # The yawing velocity of the segbot relative to the station. Positive is turning left.
v = Symbol('v')                  # The forward velocity of the segbot. Positive is moving forward.
theta = Symbol('theta')          # The pitch of the chassis of the segbot. Positive is tipped forwards.
thetadot = Symbol('thetadot')    # The pitching velocity of the segbot. Positive is tipping forward.
tau_l = Symbol('tau_l')          # The torque applied to the left wheel of the segbot. Positive moves the segbot forward.
tau_r = Symbol('tau_r')          # The torque applied to the right wheel of the segbot. Positive moves the segbot forward.

M = Matrix([[m + 2 * iaa_w / r_w**2, 0, m_c * h * cos(theta)],
            [0, (ixx + m_c * h**2) * sin(theta)**2 + izz * cos(theta)**2 + (2 * iaa_w * a**2 / r_w**2) + 2 * m_w * a**2, 0],
            [m_c * h * cos(theta), 0, iyy * m_c * h**2]])
N = Matrix([[m_c * h * (phidot**2 + thetadot**2) * sin(theta)],
            [-2 * (ixx - izz + m * h**2) * cos(theta) * sin(theta) * phidot * thetadot - m_c * h * sin(theta) * v * phidot],
            [(ixx - izz + m_c * h**2) * cos(theta) * sin(theta) * phidot**2 + m_c * 9.81 * h * sin(theta)]])
R = Matrix([[1 / r_w, 1 / r_w],
            [-a / r_w, a / r_w],
            [-1, -1]])
f_partial = simplify(M.inv() * (N + R * Matrix([[tau_l], [tau_r]])))
f = Matrix([[v * sin(phi)],
            [f_partial]])
f

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                -18.4325593081475⋅τₗ - 18.4325593081475⋅τᵣ - 14.377396260355⋅
⎢                ─────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                     89.85872
⎢                                                                             
⎢                                                  -1.056⋅φ̇⋅θ̇⋅sin(2⋅θ) - 2.4
⎢                                                  ───────────────────────────
⎢                                                                             
⎢                                                                          0.9
⎢                                                   

## System Linearization and State Space Form

In [7]:
# FIXME: Find an equilibrium point and linearize the system into state space form
# Generalized coordinates
e_lat, phi, phidot, v, theta, thetadot, tau_l, tau_r = sym.symbols('e_lat phi phidot v theta thetadot tau_l tau_r')

# Compute the Jacobians
state_vars = sym.Matrix([theta, thetadot, phi, phidot, v, e_lat])
inputs = sym.Matrix([tau_l, tau_r])

# Jacobian matrices
A = f.jacobian(state_vars)
B = f.jacobian(inputs)

# Define equilibrium point: theta = 0, phi = 0, all velocities = 0, torques = 0
equilibrium_point = {
    theta: 0,
    thetadot: 0,
    phi: 0,
    phidot: 0,
    v: 0,
    tau_l: 0,
    tau_r: 0
}

# Substitute equilibrium values into the Jacobians A and B
A_eq = A.subs(equilibrium_point)
B_eq = B.subs(equilibrium_point)

A_eq, B_eq

⎛⎡        0          0  0  0  0  0⎤  ⎡        0                  0        ⎤⎞
⎜⎢                                ⎥  ⎢                                    ⎥⎟
⎜⎢-245.056661411115  0  0  0  0  0⎥  ⎢15.5326172644856   15.5326172644856 ⎥⎟
⎜⎢                                ⎥, ⎢                                    ⎥⎟
⎜⎢        0          0  0  0  0  0⎥  ⎢-1.04916430028235  1.04916430028235 ⎥⎟
⎜⎢                                ⎥  ⎢                                    ⎥⎟
⎝⎣1592.91663381947   0  0  0  0  0⎦  ⎣-99.6830245697014  -99.6830245697014⎦⎠

## FIXME: Gain Selection and Controller Design

In [11]:
# FIXME: Select a set of control and observer gains. Ensure the resultant system is stable. Implement your controller below.

# PID controller parameters for pitch (theta) and yaw (phi)
Kp_theta = 10.0  # Proportional gain for pitch
Ki_theta = 1.0   # Integral gain for pitch
Kd_theta = 2.0   # Derivative gain for pitch

Kp_phi = 10.0    # Proportional gain for yaw
Ki_phi = 1.0     # Integral gain for yaw
Kd_phi = 2.0     # Derivative gain for yaw

# Placeholder for storing integral terms (initialize to zero)
integral_theta = 0.0
integral_phi = 0.0

# Placeholder for storing previous errors (for derivative calculation)
previous_error_theta = 0.0
previous_error_phi = 0.0

# Time step (assume a fixed time step for simplicity)
dt = 0.01  # 10 ms

# Example target setpoints for pitch and yaw
target_theta = 0.0  # Desired pitch (rad)
target_phi = 0.0    # Desired yaw (rad)

# Current states (these would be dynamically updated in real-time in an actual system)
current_theta = 0.1  # Current pitch (rad)
current_phi = 0.2    # Current yaw (rad)

# Compute errors for pitch and yaw
error_theta = target_theta - current_theta
error_phi = target_phi - current_phi

# Compute integral terms
integral_theta += error_theta * dt
integral_phi += error_phi * dt

# Compute derivative terms
derivative_theta = (error_theta - previous_error_theta) / dt
derivative_phi = (error_phi - previous_error_phi) / dt

# Compute control signals for torques
tau_l_theta = Kp_theta * error_theta + Ki_theta * integral_theta + Kd_theta * derivative_theta
tau_r_theta = Kp_theta * error_theta + Ki_theta * integral_theta + Kd_theta * derivative_theta

tau_l_phi = Kp_phi * error_phi + Ki_phi * integral_phi + Kd_phi * derivative_phi
tau_r_phi = Kp_phi * error_phi + Ki_phi * integral_phi + Kd_phi * derivative_phi

# Total torques applied to the left and right wheels (combining pitch and yaw control)
tau_l = tau_l_theta + tau_l_phi
tau_r = tau_r_theta - tau_r_phi

# Update previous errors for the next step
previous_error_theta = error_theta
previous_error_phi = error_phi

tau_l, tau_r  # Outputs of the PID controller (torques applied to the wheels)

In [16]:
# Create the Controller class
class Controller():
    def __init__(self):
        """
        Initializes the controller. You should initialize any member variables that
        the controller will use to their starting values. This function is called
        when a new instance of the Controller class is created.
    
        Parameters
        ----------
        None.
    
        Returns
        -------
        None.
        """
        # FIXME: Replace the following code with your initialization function

        # PID gains for pitch control (theta)
        self.Kp_theta = 10.0
        self.Ki_theta = 1.0
        self.Kd_theta = 2.0
        
        # PID gains for yaw control (phi)
        self.Kp_phi = 10.0
        self.Ki_phi = 1.0
        self.Kd_phi = 2.0
        
        # Integral terms
        self.integral_theta = 0.0
        self.integral_phi = 0.0
        
        # Previous errors for derivative terms
        self.previous_error_theta = 0.0
        self.previous_error_phi = 0.0
    
    def reset(self):
        """
        Resets the controller to an initial state. You should reset any variables
        that track states, times, etc. in this function. This function will be 
        called just before the simulation starts running and any time the simulation
        is reset.
    
        Parameters
        ----------
        None.
    
        Returns
        -------
        None.
        """
        # FIXME: Replace the following code with your reset function

        # Reset integral and previous error terms
        self.integral_theta = 0.0
        self.integral_phi = 0.0
        self.previous_error_theta = 0.0
        self.previous_error_phi = 0.0

    
    def run(self, **kwargs):
        """
        Runs the controller. Data is passed from the simulation environment to the
        controller via kwargs, the controller calculates the inputs to the system,
        and then returns them via a return statement
    
        Parameters
        ----------
        **kwargs : Dictionary
            A dictionary of data passed from the simulation environment to the 
            controller. Specific data is extracted from kwargs via the statement:
            kwargs["KEYWORD"], where "KEYWORD" is a key to the dictionary. The keys
            of kwargs for this project are as follows:

            kwargs["e_lat"] : Float
                The current lateral position error of the segbot in meters.
            
            kwargs["v"] : Float
                The current forward velocity of the segbot in meters/second.
            
            kwargs["phi"] : Float
                The current heading angle of the chassis in radians.
            
            kwargs["phidot"] : Float
                The current heading  angular rate of the chassis in radians/second.
            
            kwargs["theta"] : Float
                The current pitch angle of the chassis in radians.
            
            kwargs["thetadot"] : Float
                The current pitch angular rate of the chassis in radians/second.
    
        Returns
        -------
        inputs : List with length equal to the number of inputs
            This is a list of the inputs calculated by the controller. In this
            project, there are two inputs, the torque applied to the left wheel
            and the torque applied to the right wheel. Therefore, inputs has the 
            form:
            inputs = [left_torque, right_torque]
        
        """
        # FIXME: Replace the following manual controller with your own controller
        # Target setpoints (desired values)
        target_theta = 0.0  # Desired pitch
        target_phi = 0.0    # Desired yaw

        # Extract current states from kwargs
        current_theta = kwargs["theta"]
        current_phidot = kwargs["phidot"]
        current_thetadot = kwargs["thetadot"]
        current_phi = kwargs["phi"]

        # Time step (assuming a fixed time step)
        dt = 0.01  # Example value (this should ideally be dynamically set)

        # Compute errors
        error_theta = target_theta - current_theta
        error_phi = target_phi - current_phi

        # Update integral terms
        self.integral_theta += error_theta * dt
        self.integral_phi += error_phi * dt

        # Compute derivative terms
        derivative_theta = (error_theta - self.previous_error_theta) / dt
        derivative_phi = (error_phi - self.previous_error_phi) / dt

        # Compute control signals (torques) using PID
        tau_l_theta = (self.Kp_theta * error_theta +
                       self.Ki_theta * self.integral_theta +
                       self.Kd_theta * derivative_theta)
        tau_r_theta = tau_l_theta  # Both wheels are affected equally by pitch

        tau_l_phi = (self.Kp_phi * error_phi +
                     self.Ki_phi * self.integral_phi +
                     self.Kd_phi * derivative_phi)
        tau_r_phi = -tau_l_phi  # Yaw control affects wheels oppositely

        # Combine torques for pitch and yaw
        left_torque = tau_l_theta + tau_l_phi
        right_torque = tau_r_theta - tau_r_phi

        # Update previous errors for the next step
        self.previous_error_theta = error_theta
        self.previous_error_phi = error_phi

        # Return the calculated torques for the left and right wheels
        inputs = [left_torque, right_torque]
        return inputs

In [17]:
# Create an instance of our Controller class
controller = Controller()

## Running the Simulation

Before we run the simulation, we must first import the simulation module and then create an instance of the simulator. This is done below:

In [18]:
# Import the segbot project. This module is used to simulate, render, and plot a segbot dynamic system
import ae353_segbot

There are four main parameters to the Segbot_sim class initialization function:
#### Parameters

> **use_keyboard** : *bool, optional*  
> A boolean flag that indicates whether the simulation will allow the use of keyboard interactivity. The default is True.
> 
> **visualization** : *bool, optional*  
> A boolean flag that indicates whether the simulation will be  visualized in meshcat. The default is True.
> 
> **animation** : *bool, optional*  
> A boolean flag that indicates whether animated plots are created in real time. The default is True.
> 
> **bumpy**: *bool, optional*  
> A boolean flag that indicates whether the station will be built with bumps on the driving surface or not.

In [19]:
sim = ae353_segbot.Segbot_sim(use_keyboard=True,
                              visualization=True,
                              animation=False,
                              bumpy=False)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


Now we are ready to run the simulation and collect data. To do this we call the ``sim.run()`` function. This function takes an instance of the ``Controller`` class as an argument and returns ``data``. The parameters and return values of ``sim.run()`` are shown below.

#### Parameters

> **controller** : *member of Controller class*  
> Your controller that will generated inputs to the system.
> 
> **max_time** : *Float or None, optional*  
> The total amount of time the simulation is allowed to run. If set to None, the simulation will run until "ESC" is pressed on the keyboard. If the keyboard is disabled, and max_time is set to None, the simulation will automatically terminate after 10 seconds of simulated time. The default value is None.
> 
> **initial_e_lat** : *Float, optional*  
> The initial lateral error in meters. Changes the target lateral error, does not change the lateral position of the segbot. The default value is 0.0.
> 
> **station_velocity** : *Float, optional*  
> The constant rotational speed of the station in rad/s. The default value is -0.1.

#### Returns

> **data** : *Dictionary of Lists*  
> A dictionary containing all relevant data generated during the simulation. Specific data is extracted via the statement: data["KEYWORD"], where "KEYWORD" is a key to the dictionary. The keys of data for this project are as follows:
>  
> **data["lat_pos"]** : *List of Floats*  
> A list of the lateral position of the segbot in meters at each time stamp during the simulation.
>  
> **data["tag_lat"]** : *List of Floats*  
> A list of the taget lateral position of the segbot in meters at each time stamp during the simulation.  
>  
> **data["e_lat"]** : *List of Floats*  
> A list of the lateral position error of the segbot in meters at each time stamp during the simulation.
>  
> **data["v"]** : *List of Floats*
> test
>  
> A list of the forward velocity of the segbot in meters/second at each time stamp during the simulation.
> 
> **data["phi"]** : *List of Floats*  
> A list of the heading angle of the chassis in radians at each time stamp during the simulation.
> 
> **data["phidot"]** : *List of Floats*  
> A list of the heading  angular rate of the chassis in radians/second at each time stamp during the simulation.
> 
> **data["theta"]** : *List of Floats*  
> A list of the pitch angle of the chassis in radians at each time stamp during the simulation.
> 
> **data["thetadot"]** : *List of Floats*  
> A list of the pitch angular rate of the chassis in radians/second at each time stamp during the simulation.
> 
> **data["tau_l"]** : *List of Floats*  
> A list of the torque applied to the left wheel of the segbot in Newton-meters at each time stamp during the simulation.
> 
> **data["tau_r"]** : *List of Floats*  
> A list of the torque applied to the right wheel of the segbot in Newton-meters at each time stamp during the simulation.
> 
> **data["time"]** : *List of Floats*  
> A list of the time stamps in seconds.

#### If enabled you can use the keyboard to interact with the simulation:  
> press **ENTER** to start the simulation  
> press **BACKSPACE** to reset the simulation  
> press **SPACE** to pause the simulation  
> press **ESC** to end the simulation  

#### To control the target lateral position:
> press **j** to move the target to the left  
> press **l** to move the target to the right  

#### If the manual controller is being used:
> press **w** to apply small forward torque to both wheels  
> press **SHIFT+w** to apply large forward torque to both wheels  
> press **s** to apply small backward torque to both wheels  
> press **SHIFT+s** to apply large backward torque to both wheels  
> press **d** to apply small forward torque to only the left wheel  
> press **SHIFT+d** to apply large forward torque to only the left wheel  
> press **a** to apply small backward torque to only the right wheel  
> press **SHIFT+a** to apply large backward torque to only the right wheel  

In [ ]:
# Run the simulation and collect the simulation data
data = sim.run(controller,
               max_time=None,
               initial_e_lat = 0.0,
               station_velocity = -0.1)

## FIXME: Post-processing the simulation data

After the simulation is run, we can plot the data using the Matplotlib package. To start, we import matplotlib and numpy.

In [ ]:
# Import the pyplot module from the matplotlib package
import matplotlib.pyplot as plt

# Import numpy
import numpy as np

Now we can make whichever plots we see fit to make sense of the simulation data. Here we plot the pitch versus time as well as the applied torques versus time.

**THIS IS LEFT INTENTIONALLY AS INSUFFICIENT TO PROVE YOUR CONTROLLER CAN STEER THE SEGBOT.**

In [ ]:
# Create a figure that has two plots in it
fig, axes = plt.subplots(2, 1, figsize=(7.2, 5.4), sharex=True)

# Create a plot of the pitch angle versus time on the first plot
axes[0].plot(data['time'], 180.*np.array(data['theta'])/np.pi, c='k', lw=2.5)
axes[0].set_ylabel("Pitch [deg]", fontsize=12)
axes[0].tick_params(axis='y', labelsize=12)
axes[0].set_title("Pitch vs. Time", fontsize=14)
axes[0].grid()

# Create a plot of the torque versus time on the second plot
axes[1].plot(data['time'], data['tau_l'], label="Left Torque", c='r', lw=2.5)
axes[1].plot(data['time'], data['tau_r'], label="Right Torque", c='b', lw=2.5)
axes[1].set_xlabel("Time [s]", fontsize=12)
axes[1].set_ylabel("Torque [Nm]", fontsize=12)
axes[1].tick_params(axis='y', labelsize=12)
axes[1].legend(fontsize=12, shadow=True)
axes[1].set_title("Torque vs. Time", fontsize=14)
axes[1].grid()

# Show the figure
fig.tight_layout(pad=2.0)
plt.show()